In [1]:
import pandas as pd

In [7]:
df = pd.read_csv(
    'Download/2010HMDA_1.csv.gz', compression='gzip', nrows=10_000, low_memory=False
)
df.head()

,action_taken,action_taken_name,agency_code,agency_abbr,agency_name,applicant_ethnicity,applicant_ethnicity_name,applicant_income_000s,applicant_race_1,applicant_race_2,...,state_abbr,state_name,hud_median_family_income,loan_amount_000s,number_of_1_to_4_family_units,number_of_owner_occupied_units,minority_population,population,rate_spread,tract_to_msamd_income
0,1,Loan originated,5,NCUA,National Credit Union Administration,2,Not Hispanic or Latino,176.0,5,NaN,...,NC,North Carolina,50400.0,390,2535.0,1181.0,2.49,3658.0,NaN,91.000000
1,1,Loan originated,7,HUD,Department of Housing and Urban Development,2,Not Hispanic or Latino,64.0,5,NaN,...,NC,North Carolina,67200.0,171,1429.0,1321.0,8.42,3680.0,NaN,187.979996
2,1,Loan originated,7,HUD,Department of Housing and Urban Development,2,Not Hispanic or Latino,95.0,5,NaN,...,NY,New York,103600.0,409,2085.0,1630.0,10.32,6165.0,NaN,92.370003
3,1,Loan originated,3,FDIC,Federal Deposit Insurance Corporation,2,Not Hispanic or Latino,43.0,5,NaN,...,PA,Pennsylvania,54100.0,48,1203.0,842.0,2.75,3415.0,1.68,103.279999
4,1,Loan originated,1,OCC,Office of the Comptroller of the Currency,3,"Information not provided by applicant in mail,...",60.0,6,NaN,...,MO,Missouri,68300.0,160,2305.0,2195.0,3.31,5740.0,NaN,168.899994


In [8]:
df_sample = pd.read_csv(
    'Download-Sample/2010HMDA_1.csv.gz', compression='gzip', nrows=10_000, low_memory=False
)
df_sample.head()

,action_taken,action_taken_name,agency_code,agency_abbr,agency_name,applicant_ethnicity,applicant_ethnicity_name,applicant_income_000s,applicant_race_1,applicant_race_2,...,state_abbr,state_name,hud_median_family_income,loan_amount_000s,number_of_1_to_4_family_units,number_of_owner_occupied_units,minority_population,population,rate_spread,tract_to_msamd_income
0,1,Loan originated,5,NCUA,National Credit Union Administration,2,Not Hispanic or Latino,176.0,5,NaN,...,NC,North Carolina,50400.0,390,2535.0,1181.0,2.49,3658.0,NaN,91.000000
1,1,Loan originated,7,HUD,Department of Housing and Urban Development,2,Not Hispanic or Latino,64.0,5,NaN,...,NC,North Carolina,67200.0,171,1429.0,1321.0,8.42,3680.0,NaN,187.979996
2,1,Loan originated,7,HUD,Department of Housing and Urban Development,2,Not Hispanic or Latino,95.0,5,NaN,...,NY,New York,103600.0,409,2085.0,1630.0,10.32,6165.0,NaN,92.370003
3,1,Loan originated,3,FDIC,Federal Deposit Insurance Corporation,2,Not Hispanic or Latino,43.0,5,NaN,...,PA,Pennsylvania,54100.0,48,1203.0,842.0,2.75,3415.0,1.68,103.279999
4,1,Loan originated,1,OCC,Office of the Comptroller of the Currency,3,"Information not provided by applicant in mail,...",60.0,6,NaN,...,MO,Missouri,68300.0,160,2305.0,2195.0,3.31,5740.0,NaN,168.899994


In [12]:
KEEP_COLS = [
    "action_taken",
    "state_abbr",
    "respondent_id",
    "loan_amount_000s",
    "applicant_income_000s"
]

In [16]:
def _select_keep_cols(df: pd.DataFrame) -> pd.DataFrame:
    cols = [c for c in KEEP_COLS if c not in df_sample.columns]
    if cols:
        raise ValueError(f"Missing columns: {cols}")
    return df[KEEP_COLS] 

In [20]:
df_samp_filt = _select_keep_cols(df_sample)
print(df_samp_filt.dtypes)
df_samp_filt.head()

action_taken               int64
state_abbr                object
respondent_id             object
loan_amount_000s           int64
applicant_income_000s    float64
dtype: object


,action_taken,state_abbr,respondent_id,loan_amount_000s,applicant_income_000s
0,1,NC,0000000227,390,176.0
1,1,NC,1461400001,171,64.0
2,1,NY,20-0960457,409,95.0
3,1,PA,0000007799,48,43.0
4,1,MO,0000000024,160,60.0


In [19]:
df_samp_filt['action_taken'].value_counts()

action_taken
1    10000
Name: count, dtype: int64

In [22]:
df_samp_filt.isna().sum()

action_taken               0
state_abbr                49
respondent_id              0
loan_amount_000s           0
applicant_income_000s    583
dtype: int64

In [39]:
def _sanity_checks(df: pd.DataFrame) -> None:
    if  (df["applicant_income_000s"] >= 0).mean() < 0.90:
        assert  False, "More than 10% of applicant_income_000s is negative"
    for col in ["action_taken", "respondent_id"]:
        if df[col].isna().sum() > 0:
            assert False, f"Some value in {col} are missing"
    if not df["action_taken"].isin(list(range(1, 9))).all():
        assert False, "action_taken has unexpected values"  

In [36]:
df_samp_filt.describe()

,action_taken,loan_amount_000s,applicant_income_000s
count,10000.0,10000.000000,9417.000000
mean,1.0,212.373300,106.123394
std,0.0,166.409785,146.700982
min,1.0,1.000000,1.000000
25%,1.0,118.000000,53.000000
50%,1.0,177.000000,81.000000
75%,1.0,266.000000,122.000000
max,1.0,5500.000000,8959.000000


In [33]:
df_sample.to_csv('sample_2010HMDA.csv.gz', compression='gzip', index=False)
df_samp_filt.to_csv('filt_2010HMDA.csv.gz', compression='gzip', index=False)

In [47]:
import os

In [51]:
FILE_PATH = 'Download/'
for file in os.listdir(FILE_PATH):
    print(file)

2010HMDA_1.csv.gz
2010HMDA_2.csv.gz
2010HMDA_3.csv.gz
2010HMDA_4.csv.gz
2010HMDA_5.csv.gz
2010HMDA_6.csv.gz
2010HMDA_7.csv.gz


In [54]:
FILE_PATH = 'Download/'
create_file = True
for n, file in enumerate(os.listdir(FILE_PATH)):
    for i, df in enumerate(pd.read_csv(os.path.join(FILE_PATH, file), 
                                       compression='gzip', 
                                       chunksize=100_000, 
                                       low_memory=False)):
        df_filt = _select_keep_cols(df)
        print(f"File {n+1} | Chunk {i+1} - {df.shape}")
        _sanity_checks(df_filt)
        df_filt.to_csv(f'clean_2010HMDA.csv.gz', 
                    mode=('w' if create_file else 'a'),
                    compression='gzip', 
                    header=create_file,
                    index=False)
        create_file = False

File 1 | Chunk 1 - (100000, 78)
File 1 | Chunk 2 - (100000, 78)
File 1 | Chunk 3 - (100000, 78)
File 1 | Chunk 4 - (100000, 78)
File 1 | Chunk 5 - (100000, 78)
File 1 | Chunk 6 - (100000, 78)
File 1 | Chunk 7 - (100000, 78)
File 1 | Chunk 8 - (100000, 78)
File 1 | Chunk 9 - (100000, 78)
File 1 | Chunk 10 - (100000, 78)
File 2 | Chunk 1 - (100000, 78)
File 2 | Chunk 2 - (100000, 78)
File 2 | Chunk 3 - (100000, 78)
File 2 | Chunk 4 - (100000, 78)
File 2 | Chunk 5 - (100000, 78)
File 2 | Chunk 6 - (100000, 78)
File 2 | Chunk 7 - (100000, 78)
File 2 | Chunk 8 - (100000, 78)
File 2 | Chunk 9 - (100000, 78)
File 2 | Chunk 10 - (100000, 78)
File 3 | Chunk 1 - (100000, 78)
File 3 | Chunk 2 - (100000, 78)
File 3 | Chunk 3 - (100000, 78)
File 3 | Chunk 4 - (100000, 78)
File 3 | Chunk 5 - (100000, 78)
File 3 | Chunk 6 - (100000, 78)
File 3 | Chunk 7 - (100000, 78)
File 3 | Chunk 8 - (100000, 78)
File 3 | Chunk 9 - (100000, 78)
File 3 | Chunk 10 - (100000, 78)
File 4 | Chunk 1 - (100000, 78)
File 

In [55]:
pd.read_csv('clean_2010HMDA.csv.gz', compression='gzip', nrows=10).shape

(10, 5)